In [1]:
# warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import openpyxl
from tqdm import tqdm
import unidecode
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import warnings
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import math
from operator import itemgetter

In [28]:
alc = pd.read_excel("ALCALDIAS_FINAL.xlsx")
alc = alc[alc.date == 2015]

In [29]:
df = pd.read_stata("PRUEBA_REG_DIS.dta")

In [30]:
df = df.drop(columns=list(alc.columns[4:]))
df = df.iloc[:, :11]

In [34]:
df = df.merge(alc, "left", "dane")

In [37]:
df.to_csv("PRUEBA_REG_DIS2.csv")

In [30]:
#Función para encontrar la coincidencia de nombres con un puntaje mínimo
def match_name(name, list_names, min_score=0):
    max_score = -1 #Inicializa la puntuación máxima con -1
    max_name = "" #Inicializa el nombre máximo con una cadena vacía
    for name2 in list_names:
        score = process.extractOne(name, [name2])[1] #Calcula la puntuación de coincidencia entre 'name' y 'name2'
        if (score > min_score) & (score > max_score): #Si la puntuación es mayor que el puntaje mínimo y mayor que el puntaje máximo
            max_name = name2 #Actualiza el nombre máximo
            max_score = score #Actualiza la puntuación máxima
    return max_name, max_score #Retorna el nombre máximo y la puntuación máxima
    
def dane_dpto(df):
    dict_list = []
    for name in df['dpto']:
        match = match_name(name, dane_codes.keys(), 50) #Encuentra la coincidencia del nombre con los códigos dane

        dict_ = {}
        dict_.update({"dpto" : name}) 
        dict_.update({"match_name" : match[0]}) #Actualiza el nombre de la coincidencia
        dict_.update({"score" : match[1]}) #Actualiza la puntuación de la coincidencia
        dict_list.append(dict_) #Añade el diccionario a la lista
        
    merge_table = pd.DataFrame(dict_list)

    df['dane'] = merge_table['match_name'].map(dane_codes) #Mapea el nombre de la coincidencia con los códigos dane

    return df #Retorna el dataframe

dane_codes = {
    'Amazonas': '91',
    'Antioquia': '05',
    'Arauca': '81',
    'Atlántico': '08',
    'Bolívar': '13',
    'Boyacá': '15',
    'Caldas': '17',
    'Caquetá': '18',
    'Casanare': '85',
    'Cauca': '19',
    'Cesar': '20',
    'Chocó': '27',
    'Córdoba': '23',
    'Cundinamarca': '25',
    'Guainía': '94',
    'Guaviare': '95',
    'Huila': '41',
    'La Guajira': '44',
    'Magdalena': '47',
    'Meta': '50',
    'Nariño': '52',
    'Norte de Santander': '54',
    'Putumayo': '86',
    'Quindío': '63',
    'Risaralda': '66',
    'San Andrés y Providencia': '88',
    'Santander': '68',
    'Sucre': '70',
    'Tolima': '73',
    'Valle del Cauca': '76',
    'Vaupés': '97',
    'Vichada': '99',
    'Bogotá, D.C.': '11'
}

In [57]:
alc = pd.read_excel("ALCALDIAS_FINAL.xlsx")
alc19 = alc[alc.date==2019]

In [58]:
gob = pd.read_excel("GOBERNACIONES_FINAL.xlsx")
gob19 = gob[gob.date==2019]
gob19["date"] = gob19["date"].astype(int)
gob19["dane"] = gob19["dane"].astype(int)

In [59]:
poblacion = pd.read_stata("CEDE//poblacion.dta")
poblacion.columns = ["dane", "dpto", "municipio", "date", "population"]
poblaciondpt = poblacion.groupby(["dpto", "date"]).sum().reset_index()

In [60]:
poblaciondpt = dane_dpto(poblaciondpt)
poblaciondpt["date"] = poblaciondpt["date"].astype(int)
poblaciondpt["dane"] = poblaciondpt["dane"].astype(int)

In [61]:
gob19 = pd.merge(gob19, poblaciondpt[["dane", "date", "population"]], "left", ["dane", "date"])

In [62]:
alc19 = pd.merge(alc19, poblacion[["dane", "date", "population"]], "left", ["dane", "date"])

In [63]:
entidades = pd.concat([alc19,gob19])

In [73]:
deuda = pd.read_excel("DEUDA.xlsx")
deuda = deuda[['id', 'entidad', 'tipo_entidad', 'id_dpt', 'año',
       'saldo_deuda($) ']]
deuda.columns = ["dane", "entidad", "tipo", "id_dpto", "date", "deuda"]

In [74]:
deuda_fv = pd.merge(deuda, entidades.drop(columns = ["date"]), "left", ["dane"])

In [76]:
deuda_fv = deuda_fv[deuda_fv.date>=2015]

In [85]:
deuda_fv = deuda_fv[(deuda_fv.tipo=="Departamento") | (deuda_fv.tipo=="Municipio")]

In [86]:
deuda_fv["deuda_pc"] = deuda_fv["deuda"]/deuda_fv["population"]

In [88]:
deuda_fv.to_csv("BASE3.csv", index = False)

### Series de deuda

In [14]:
deuda = pd.read_excel("DEUDA.xlsx")
deuda = deuda[['id', 'entidad', 'tipo_entidad', 'id_dpt', 'año',
       'saldo_deuda($) ']]
deuda.columns = ["dane", "entidad", "tipo", "id_dpto", "date", "deuda"]

### Series de ingresos

In [15]:
ingresos = pd.read_stata("CEDE//PANEL_BUEN_GOBIERNO(2022).dta")
Ys = ["deficit_total", "g_cap_resto", "g_cap_FBKF", "g_cap",  "y_corr", "DF_ing_func", "DF_ing_propios"]
ingresos = ingresos[["codmpio", "ano"] + Ys]
ingresos.columns = ["dane", "date", "deficit_total", "g_cap_resto", "g_cap_fbkf", "g_cap", "y_corr", "df_ing_func", "df_ing_propios"]

In [44]:
ingresos["df_ing_func"] = ingresos["df_ing_func"]/100
ingresos["df_ing_propios"] = ingresos["df_ing_propios"]/100

ingresos["y_corr_func"]    = ingresos["df_ing_func"]    * ingresos["y_corr"]
ingresos["y_corr_propios"] = ingresos["df_ing_propios"] * ingresos["y_corr"]

In [56]:
df_temp = pd.merge(deuda, ingresos,on=['dane', 'date'])

### Población

In [18]:
poblacion = pd.read_stata("CEDE//poblacion.dta")
poblacion.columns = ["dane", "dpto", "municipio", "date", "population"]
# poblacion = poblacion[["dane", "date", "population"]]

# poblacion = poblacion[poblacion.date==2015]
# poblacion = poblacion[["dane", "population"]]

### Pegue de series

In [57]:
dff = pd.merge(df_temp, poblacion,on=['dane'])

In [58]:
Yiniciales = ['deuda', 'deficit_total','g_cap_resto', 'g_cap_fbkf', 'g_cap', 'y_corr', 'y_corr_func', 'y_corr_propios']

In [59]:
for i in Yiniciales:
    dff[f"{i}_pc"] = dff[i]/dff["population"]

In [64]:
dff = dff[(dff.date>=2012) & (dff.date<=2019)]

In [65]:
dff = pd.merge(dff, alc19.drop(columns = ["date"]), "left", "dane")

In [66]:
dff.to_csv("PRUEBA2.csv", index = False)

In [24]:
poblaciondpt = poblacion.groupby(["dpto", "date"]).sum().reset_index()

### Regresiones

In [ ]:
import statsmodels.api as sm
from tqdm import tqdm
import seaborn as sns
import statsmodels.formula.api as smf
from scipy.stats import norm

In [ ]:
Yn = ["DEUDA", "deficit_total", "g_cap_resto", "g_cap_FBKF", "g_cap",  "y_corr"]
Yp = ["DINGP", "DINGF", "DING", "DF_ing_func", "DF_ing_propios"]

In [ ]:
for año in [2015,2016,2017,2019,2020,2021]:
    dfr[f"T_{año}"] = (dfr["AÑO"]>=año).astype(int) * dfr["T1"]

<ipython-input-609-b50f1a03cbac>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfr[f"T_{año}"] = (dfr["AÑO"]>=año).astype(int) * dfr["T1"]
<ipython-input-609-b50f1a03cbac>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfr[f"T_{año}"] = (dfr["AÑO"]>=año).astype(int) * dfr["T1"]
<ipython-input-609-b50f1a03cbac>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https

In [ ]:
Yn = ["DEUDA"]
Yn = ["DEUDA", "deficit_total", "g_cap_resto", "g_cap_FBKF", "g_cap",  "y_corr"]

In [ ]:
p = []
for y in Yp:

    # dfr[f"{y}_PC"] = dfr[f"{y}"]/dfr["POBLACION"]
    dfr[f"{y}_PC"] = dfr[f"{y}"].copy()
    data_f = dfr.dropna(subset=[f"{y}_PC", "T1", "P2019", "ID_DPTO"])
    data_f = data_f[(data_f["AÑO"] > 2015) & (data_f["DIFERENCIA_GANADOR19"]<1) & (data_f["POBLACION"]>100000)]
    data_f = data_f.dropna(subset=[f"{y}", "T1", "P2019", "ID_DPTO"])
    formula = f"{y}_PC~ T1*P2019"  
    model = smf.ols(formula, data=data_f).fit()
    model = model.get_robustcov_results(cov_type='cluster', groups=data_f['ID_DPTO'])
    coef = model.params[-1]
    pvalue = model.pvalues[-1]

    p.append(pvalue)
    if pvalue < 0.1:
        print(f"{y}_PC", "**", coef, "***", model.params[0])


DINGP_PC ** -0.37091978712196516 *** 0.3573610690364498
DINGF_PC ** -0.6493295513931965 *** 0.7138385854653307
DING_PC ** -0.2883477856346563 *** 0.301856668088509
DF_ing_func_PC ** 4.321006642323391 *** 47.535932263155686
DF_ing_propios_PC ** 1.680907080525125 *** 84.03779769478793


<ipython-input-729-eb699b5ada2b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfr[f"{y}_PC"] = dfr[f"{y}"].copy()
<ipython-input-729-eb699b5ada2b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfr[f"{y}_PC"] = dfr[f"{y}"].copy()
<ipython-input-729-eb699b5ada2b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gui